In [ ]:
%pylab inline
import os
import shutil
from glob import glob
import pysumma as ps
import xarray as xr
import pandas as pd
from pathlib import Path

In [ ]:
src_params = glob('../sites/**/best_calibration/params/parameter_trial.nc', recursive=True)
sites = [p.split('/')[2] for p in src_params]
bad_sites = []
sim_sites = [s for s in sites if s not in bad_sites]

seed = 50334
np.random.seed(seed)
np.random.shuffle(sim_sites)
len(sim_sites)

nfolds = 4
kfold_test_sites = np.array(sim_sites).reshape(nfolds, -1)
kfold_train_sites = np.vstack([
    list(set(sim_sites) - set(train_sites)) for train_sites in kfold_test_sites
])
kfold_train_sites = np.vstack([
    list(set(sim_sites) - set(train_sites)) for train_sites in kfold_train_sites
])
model_src = [f'../models/train_inputs_set_{n}.txt' for n in range(nfolds)]
model_rel_dest = [f'../params/train_inputs_set_{n}.txt' for n in range(nfolds)]

executable = '../ml_summa/bin/ml_summa'

kfold_configs = []
for n in range(nfolds) :
    kfold_sites = kfold_train_sites[n]
    config = {site: {'file_manager': f'../sites/{site}/file_manager.txt'} for site in kfold_sites}
    
    for s, c in config.items():
        # set model in file manager
        fman = ps.FileManager(c['file_manager'], name='')
        fman.options.append(ps.file_manager.FileManagerOption('neuralNetFile', model_rel_dest[n]))
        fman['outFilePrefix'] = f'nn_{n}_output'
        
        # save file manager
        fman.write(path=str(fman.original_path).replace('file_manager', f'file_manager_{n}_NN'))
        
        # copy neural net file over to params
        nn_dest = c['file_manager'].replace('file_manager.txt', model_rel_dest[n].replace('../', ''))
        shutil.copy(model_src[n], nn_dest)
        c['file_manager'] = f'../sites/{s}/file_manager_{n}_NN.txt'
    kfold_configs.append(config)

In [ ]:
def spinup(sim, name, run_option, period, niters, kwargs={}):
    sim.initialize()
    sim.apply_config(kwargs.get('config', {}))
    sim.spinup(run_option, period=period, niters=niters, run_suffix=f'pysumma_spinup_{name}', **kwargs)
    sim.process = None
    return sim

In [ ]:
all_summaries = []
all_ens = []
failed = []
for i in range(nfolds):
    print(i)
    config = kfold_configs[i]
    ens = ps.Ensemble(executable, config, num_workers=22)
    ens.map(spinup, ('local', '365D', 10))
    summary = ens.summary()
    all_summaries.append(summary)
    all_ens.append(ens)

In [ ]:
failed = [s for ss in all_summaries for s in ss['error']]
success = [s for ss in all_summaries for s in ss['success']]

failed

len(failed), len(success)

In [ ]:
for s in sites:
    orig_ic = f'../sites/{s}/params/initial_conditions.nc'
    back_ic = f'../sites/{s}/params/coldstate_ml_flux.nc'
    new_ic = f'../sites/{s}/params/ic_ml_flux.nc'
    spin_ic = glob(f'../sites/{s}/params/*restart*.nc')[0]
    shutil.copy2(orig_ic, back_ic)
    shutil.copy2(spin_ic, orig_ic)